# LCL test

In [1]:
import metpy.calc as calc
import metpy._calc_mod as calc_mod
from metpy.units import units
import numpy as np

In [2]:
shape = (50, 800, 800)

# Generate synthetic temperature (in degC) with a realistic gradient
T_C = np.random.normal(loc=25.0, scale=5.0, size=shape)  # e.g., mean 25°C, stddev 10

# Generate dew point temperatures a few degrees lower than temperature
dewT_C = T_C - np.abs(np.random.normal(loc=3.0, scale=2.0, size=shape))  # always <= T_C

# Generate pressure values in hPa, decreasing with altitude
z = np.linspace(0, 8, shape[0])  # approx. 0 to 12 km altitude
P_levels = 1013.25 * np.exp(-z / 7.0)  # simple exponential decay
P_hPa = P_levels[:, None, None] * np.ones((1, shape[1], shape[2]))

In [3]:
# save the data
np.savez('data.npz', T_C=T_C, dewT_C=dewT_C, P_hPa=P_hPa)

In [3]:
# load the data
data = np.load('data.npz')

T_C = data['T_C']
dewT_C = data['dewT_C']
P_hPa = data['P_hPa']

## Original LCL in Python

In [4]:
%%time
p_lcl, t_lcl = calc.lcl(P_hPa * units.hPa, T_C * units.degC, dewT_C * units.degC)

CPU times: user 9.96 s, sys: 845 ms, total: 10.8 s
Wall time: 11 s


In [6]:
print(p_lcl[:, 400, 400])
print('\n')
print(t_lcl[:, 400, 400])

[951.153573816652 989.1944898420074 946.6832526346772 902.0721180469827 890.1661293868065 843.6444404458566 841.8376895352983 808.5338690856872 774.520937986129 812.9349463531141 773.8117437404695 768.1940174696426 721.7861116323601 716.382007064664 711.4717205953177 669.5837345020277 646.6572385769028 678.5599939688796 645.6435114243565 647.4110005513435 611.1274149210313 612.4685826103347 567.5442244992857 546.7451513144853 553.8672269332386 557.5438709278977 539.5529538618025 526.965383201809 492.5482768155783 471.2649364340462 466.05814841805284 463.46562378761126 453.3871261214159 416.07086602094546 432.646798386456 428.15205763180717 419.39421739826236 400.6908230723861 381.99411721648124 404.3197832440895 380.81069433718875 368.6329429411107 374.5870738731854 370.50785049755297 340.53350906343934 326.76139058519067 341.98729173670046 310.74403541194397 291.8358487737801 297.5371721701187] hectopascal


[16.759164760033173 32.92897916206164 19.947631724967664 14.613826676781855 1

In [5]:
idx_nan = np.argwhere(np.isnan(p_lcl))
idx_nan

array([], shape=(0, 3), dtype=int64)

In [8]:
print("T: ", T_C[40,  32, 504], "degC")
print("dewT: ", dewT_C[40,  32, 504], "degC")
print("P: ", P_hPa[40,  32, 504], "hPa")

T:  77.64458546365549 degC
dewT:  76.46342469872707 degC
P:  398.60607922080777 hPa


## LCL manual vectorization in c++ pybind11

In [6]:
%%time
p_lcl, t_lcl = calc.lcl_linfel(P_hPa * units.hPa, T_C * units.degC, dewT_C * units.degC)

CPU times: user 5.9 s, sys: 298 ms, total: 6.2 s
Wall time: 6.2 s


In [8]:
print(p_lcl[:, 400, 400])
print('\n')
print(t_lcl[:, 400, 400])

[951.153573816652 989.1944898420074 946.6832526346786 902.0721180469827 890.1661293868057 843.6444404458562 841.8376895352983 808.5338690856872 774.5209379861296 812.9349463531141 773.8117437404695 768.1940174696426 721.7861116323592 716.382007064664 711.4717205953177 669.5837345020277 646.6572385769028 678.5599939688796 645.6435114243562 647.4110005513435 611.1274149210313 612.4685826103347 567.5442244992857 546.7451513144853 553.8672269332386 557.5438709278973 539.5529538618025 526.965383201809 492.54827681557845 471.2649364340462 466.05814841805284 463.46562378761126 453.3871261214159 416.07086602094546 432.646798386456 428.15205763180717 419.39421739826236 400.6908230723861 381.9941172164807 404.3197832440895 380.81069433718875 368.6329429411107 374.5870738731854 370.50785049755297 340.53350906343934 326.76139058519067 341.98729173670046 310.74403541194397 291.8358487737801 297.5371721701187] hectopascal


[16.759164760033173 32.92897916206164 19.947631724967778 14.613826676781855 

## LCL manual vectorization in Python

In [3]:
p_lcl, t_lcl = calc.lcl_manual(P_hPa * 100, T_C+273.15, dewT_C+273.15)

Warning in function 'SaturationMixingRatio': Total pressure must be greater than the saturation vapor pressure for liquid water to be in equilibrium.
Warning in function 'lambert_wm1': lambert_wm1 did not converge.
Warning in function 'SaturationMixingRatio': Total pressure must be greater than the saturation vapor pressure for liquid water to be in equilibrium.
Warning in function 'lambert_wm1': lambert_wm1 did not converge.
Warning in function 'SaturationMixingRatio': Total pressure must be greater than the saturation vapor pressure for liquid water to be in equilibrium.
Warning in function 'lambert_wm1': lambert_wm1 did not converge.
Warning in function 'SaturationMixingRatio': Total pressure must be greater than the saturation vapor pressure for liquid water to be in equilibrium.
Warning in function 'lambert_wm1': lambert_wm1 did not converge.


#### time used: 21 s